# Wicliff Tah
Food Wine Pairing with NLP

In [2]:
!pip install gensim
import pandas as pd
import numpy as  np
import matplotlib.pyplot as plt
import string
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import  word_tokenize, sent_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from collections import Counter, OrderedDict
from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA

[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


In [3]:
df_food = pd.read_csv('Reviews4.csv')
df_wine = pd.read_csv('all_scraped_wine_info_chardonnay.csv')

In [4]:
df_food.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,341073,B000O2EV8G,A18EYTHURMGN8N,SQ,0,0,5,1290124800,Very Good,This beef jerky is absolutely delicious. It IS...
1,341074,B000O2EV8G,AM9P4Q63GYNT6,"Matthew Fallucca ""Matt""",0,0,5,1289174400,"Great, Organic Beef Jerky!",This stuff is delicious...I had been looking f...
2,341075,B000O2EV8G,A1UFQCEVFJRSYK,G. Draper,0,0,4,1286150400,Great Organic Food,"We've tried many organic jerkys, but this has ..."
3,341076,B000O2EV8G,A27OPNET5OKMH9,Peggy Sue Leggett,0,0,5,1255824000,another winner,this product is great tasting and healthy for ...
4,341077,B000O2EV8G,A17O9AHKHK66AI,L. Gildart,0,0,5,1255305600,Expensive Tastes,My son really loves the Organic Beef Jerky. I...


In [5]:
df_wine.head()

,Unnamed: 0,Alcohol,Appellation,Bottle Size,Category,Country,Date Published,Description,Designation,Importer,...,Province,Rating,Region,Reviewer,Reviewer Twitter Handle,Subregion,User Avg Rating,Variety,Vintage,Winery
0,791,9%,"Burgenland, Austria",375 ml,Dessert,Austria,12/1/2018,"Marmalade, toffee, vanilla and tonka bean swir...",Nouvelle Vague Trockenbeerenauslese Nummer 7,Terlato Wines International,...,Burgenland,98,NaN,Anne KrebiehlÂ MW,@AnneInVino,NaN,Not rated yet [Add Your Review],Chardonnay,2015.0,Kracher
1,792,10.50%,"Burgenland, Austria",375 ml,Dessert,Austria,12/1/2018,A touch of creaminess on the nose leads to a m...,Trockenbeerenauslese Grand Selection,GD Imports,...,Burgenland,97,NaN,Anne KrebiehlÂ MW,@AnneInVino,NaN,Not rated yet [Add Your Review],Chardonnay,2015.0,H.& C. Nittnaus
2,796,9%,"Burgenland, Austria",375 ml,Dessert,Austria,12/1/2018,A smokiness lies thickly above the apple fruit...,Trockenbeerenauslese,Blue Danube Wine Co,...,Burgenland,96,NaN,Anne KrebiehlÂ MW,@AnneInVino,NaN,Not rated yet [Add Your Review],Chardonnay,2015.0,Rosenhof
3,800,10%,"Burgenland, Austria",375 ml,Dessert,Austria,12/1/2018,"Lime blossom tea, candied apple and an allurin...",Beerenauslese,Magellan Wine Imports,...,Burgenland,95,NaN,Anne KrebiehlÂ MW,@AnneInVino,NaN,Not rated yet [Add Your Review],Chardonnay,2015.0,Gunter Triebaumer
4,877,10%,"Sonoma Mountain, Sonoma, California, US",375 ml,Dessert,US,12/1/2018,"Made in a Sauternes-like method, this dessert ...",Zahav Late Harvest Botrytis,NaN,...,California,90,Sonoma,Virginie Boone,@vboone,Sonoma Mountain,Not rated yet [Add Your Review],Chardonnay,2016.0,Covenant


In [6]:
df_wine['Variety'].unique()

array(['Chardonnay', nan], dtype=object)

In [7]:
df_wine.duplicated().sum()

0

In [8]:
df_wine.isnull().sum()

Unnamed: 0                    0
Alcohol                    1853
Appellation                   9
Bottle Size                   0
Category                      0
Country                       9
Date Published                0
Description                  75
Designation                4900
Importer                   6799
Name                          0
Price                      1002
Province                     10
Rating                        0
Region                     2157
Reviewer                      0
Reviewer Twitter Handle    1674
Subregion                  8879
User Avg Rating               0
Variety                       9
Vintage                     406
Winery                       10
dtype: int64

# Training our Word Embeddings
First, we need to train a Word2Vec model on all the words in our corpus. We will process our wine and food terms separately - some of the wine terms will be standardized to account for commonalities in the colorful language of the world of wine

In [9]:
food_list = list(df_food['Text'])
wine_list = list(df_wine['Description'])

In [10]:
full_food = [str(r) for r in food_list]
food_corpus = ''.join(full_food)
food_sent_tok = sent_tokenize(food_corpus)
full_wine = [str(r) for r in wine_list]
wine_corpus = ''.join(full_wine)
wine_sent_tok = sent_tokenize(wine_corpus)

In [11]:
print(food_sent_tok[:2])

['This beef jerky is absolutely delicious.', 'It IS a little pricey, but amazon carries it for 30% less than what stores sell it for here.']


In [12]:
print(wine_sent_tok[:2])

['Marmalade, toffee, vanilla and tonka bean swirl on the nose.', 'Rich and smooth, rounded and mellow, yet with an incredibly bright, aromatic core of fine citrus.']


In [13]:
stop_words = set(stopwords.words('english')) 

punctuation_table = str.maketrans({key: None for key in string.punctuation})
sno = SnowballStemmer('english')

def normalize_text(raw_text):
    try:
        word_list = word_tokenize(raw_text)
        normalized_sentence = []
        for w in word_list:
            try:
                w = str(w)
                lower_case_word = str.lower(w)
                stemmed_word = sno.stem(lower_case_word)
                no_punctuation = stemmed_word.translate(punctuation_table)
                if len(no_punctuation) > 1 and no_punctuation not in stop_words:
                    normalized_sentence.append(no_punctuation)
            except:
                continue
        return normalized_sentence
    except:
        return ''

normalized_wine_sentences = []
for s in wine_sent_tok:
    normalized_text = normalize_text(s)
    normalized_wine_sentences.append(normalized_text)

normalized_food_sentences = []
for s in food_sent_tok:
    normalized_text = normalize_text(s)
    normalized_food_sentences.append(normalized_text)

In [14]:
# first, take care of the wine trigrams
wine_bigram_model = Phrases(normalized_wine_sentences, min_count=100)
wine_bigrams = [wine_bigram_model[line] for line in normalized_wine_sentences]
wine_trigram_model = Phrases(wine_bigrams, min_count=50)
phrased_wine_sentences = [wine_trigram_model[line] for line in wine_bigrams]
wine_trigram_model.save('wine_trigrams.pkl')

In [15]:
### now, do the same for food
food_bigram_model = Phrases(normalized_food_sentences, min_count=100)
food_bigrams = [food_bigram_model[sent] for sent in normalized_food_sentences]
food_trigram_model = Phrases(food_bigrams, min_count=50)
phrased_food_sentences = [food_trigram_model[sent] for sent in food_bigrams]
food_trigram_model.save('food_trigrams.pkl')

In [16]:
wine_trigram_model = Phraser.load('wine_trigrams.pkl')
food_trigram_model = Phraser.load('food_trigrams.pkl')

In [17]:
descriptor_mapping = pd.read_csv('descriptor_mapping.csv', encoding='latin1').set_index('raw descriptor')

def return_mapped_descriptor(word, mapping):
    if word in list(mapping.index):
        normalized_word = mapping.at[word, 'level_3']
        return normalized_word
    else:
        return word

normalized_wine_sentences = []
for sent in phrased_wine_sentences:
    normalized_wine_sentence = []
    for word in sent:
        normalized_word = return_mapped_descriptor(word, descriptor_mapping)
        normalized_wine_sentence.append(str(normalized_word))
    normalized_wine_sentences.append(normalized_wine_sentence)

In [18]:
aroma_descriptor_mapping = descriptor_mapping.loc[descriptor_mapping['type'] == 'aroma']
normalized_food_sentences = []
for sent in phrased_food_sentences:
    normalized_food_sentence = []
    for word in sent:
        normalized_word = return_mapped_descriptor(word, aroma_descriptor_mapping)
        normalized_food_sentence.append(str(normalized_word))
    normalized_food_sentences.append(normalized_food_sentence)

In [19]:
normalized_sentences = normalized_wine_sentences + normalized_food_sentences

In [20]:
wine_word2vec_model = Word2Vec(normalized_sentences, vector_size=300, min_count=8, epochs=15)
print(wine_word2vec_model)

wine_word2vec_model.save('food_word2vec_modell.bin')

Word2Vec<vocab=15853, vector_size=300, alpha=0.025>


In [21]:
wine_word2vec_model = Word2Vec.load("food_word2vec_modell.bin")

# Preprocessing The Wine Dataset

In [22]:
wine_df_clean = df_wine.copy()
order_of_geographies = ['Subregion', 'Region', 'Province', 'Country']

# replace any nan values in the geography columns with the word none
def replace_nan_for_zero(value):
    if str(value) == '0' or str(value) == 'nan':
        return 'none'
    else:
        return value

for o in order_of_geographies:
    wine_df_clean[o] = wine_df_clean[o].apply(replace_nan_for_zero)

wine_df_clean.loc[:, order_of_geographies].fillna('none', inplace=True)

In [23]:
variety_geo = wine_df_clean.groupby(['Variety', 'Country', 'Province', 'Region', 'Subregion']).size().reset_index().rename(columns={0:'count'})
variety_geo_sliced = variety_geo.loc[variety_geo['count'] > 1]

vgeos_df = pd.DataFrame(variety_geo_sliced, columns=['Variety', 'Country', 'Province', 'Region', 'Subregion', 'count']) 
vgeos_df.to_csv('chardonnayy_all_geos.csv')

In [24]:
vgeos_df.head()

,Variety,Country,Province,Region,Subregion,count
0,Chardonnay,Argentina,Mendoza Province,Agrelo,none,2
3,Chardonnay,Argentina,Mendoza Province,LujÃ¡n de Cuyo,none,11
4,Chardonnay,Argentina,Mendoza Province,MaipÃº,none,3
5,Chardonnay,Argentina,Mendoza Province,Mendoza,none,409
6,Chardonnay,Argentina,Mendoza Province,Perdriel,none,5


In [25]:
wine_df_clean.head()

,Unnamed: 0,Alcohol,Appellation,Bottle Size,Category,Country,Date Published,Description,Designation,Importer,...,Province,Rating,Region,Reviewer,Reviewer Twitter Handle,Subregion,User Avg Rating,Variety,Vintage,Winery
0,791,9%,"Burgenland, Austria",375 ml,Dessert,Austria,12/1/2018,"Marmalade, toffee, vanilla and tonka bean swir...",Nouvelle Vague Trockenbeerenauslese Nummer 7,Terlato Wines International,...,Burgenland,98,none,Anne KrebiehlÂ MW,@AnneInVino,none,Not rated yet [Add Your Review],Chardonnay,2015.0,Kracher
1,792,10.50%,"Burgenland, Austria",375 ml,Dessert,Austria,12/1/2018,A touch of creaminess on the nose leads to a m...,Trockenbeerenauslese Grand Selection,GD Imports,...,Burgenland,97,none,Anne KrebiehlÂ MW,@AnneInVino,none,Not rated yet [Add Your Review],Chardonnay,2015.0,H.& C. Nittnaus
2,796,9%,"Burgenland, Austria",375 ml,Dessert,Austria,12/1/2018,A smokiness lies thickly above the apple fruit...,Trockenbeerenauslese,Blue Danube Wine Co,...,Burgenland,96,none,Anne KrebiehlÂ MW,@AnneInVino,none,Not rated yet [Add Your Review],Chardonnay,2015.0,Rosenhof
3,800,10%,"Burgenland, Austria",375 ml,Dessert,Austria,12/1/2018,"Lime blossom tea, candied apple and an allurin...",Beerenauslese,Magellan Wine Imports,...,Burgenland,95,none,Anne KrebiehlÂ MW,@AnneInVino,none,Not rated yet [Add Your Review],Chardonnay,2015.0,Gunter Triebaumer
4,877,10%,"Sonoma Mountain, Sonoma, California, US",375 ml,Dessert,US,12/1/2018,"Made in a Sauternes-like method, this dessert ...",Zahav Late Harvest Botrytis,NaN,...,California,90,Sonoma,Virginie Boone,@vboone,Sonoma Mountain,Not rated yet [Add Your Review],Chardonnay,2016.0,Covenant


In [26]:
geographies = ['Subregion', 'Region', 'Province', 'Country']

for geo in geographies:
    wine_df_clean[geo] = wine_df_clean[geo].apply(lambda x : str(x).strip())


In [27]:
variety_geo_df = pd.read_csv('varieties_all_geos_normalized.csv', index_col=0)
variety_geo_df[variety_geo_df['Variety']=='Chardonnay']

,Variety,Country,Province,Region,Subregion,count,geo_normalized
1500,Chardonnay,Argentina,Mendoza Province,Agrelo,none,2,"Mendoza, Argentina"
1503,Chardonnay,Argentina,Mendoza Province,LujÃÂ¡n de Cuyo,none,10,"Mendoza, Argentina"
1504,Chardonnay,Argentina,Mendoza Province,MaipÃÂº,none,3,"Mendoza, Argentina"
1505,Chardonnay,Argentina,Mendoza Province,Mendoza,none,407,"Mendoza, Argentina"
1506,Chardonnay,Argentina,Mendoza Province,Perdriel,none,5,"Mendoza, Argentina"
...,...,...,...,...,...,...,...
2083,Chardonnay,US,Washington,Washington Other,Columbia Gorge (WA),26,"Columbia Valley, Washington, USA"
2084,Chardonnay,US,Washington,Washington Other,Washington,120,NaN
2085,Chardonnay,US,Washington-Oregon,none,none,3,NaN
2086,Chardonnay,Ukraine,Ukraine,none,none,2,NaN


In [28]:
wine_df_merged = pd.merge(left=wine_df_clean, right=variety_geo_df, left_on=['Variety', 'Country', 'Province', 'Region', 'Subregion'],
                         right_on=['Variety', 'Country', 'Province', 'Region', 'Subregion'])
wine_df_merged.drop(['Unnamed: 0', 'Appellation', 'Bottle Size', 'Category', 'Country', 
                     'Date Published', 'Designation', 'Importer', 'Province', 'Rating', 
                     'Region', 'Reviewer', 'Reviewer Twitter Handle', 'Subregion', 'User Avg Rating', 'Winery', 'count'], 
                    axis=1, inplace=True)
wine_df_merged.shape

(13574, 7)

In [29]:

wine_df_merged.head()

,Alcohol,Description,Name,Price,Variety,Vintage,geo_normalized
0,9%,"Marmalade, toffee, vanilla and tonka bean swir...",Kracher 2015 Nouvelle Vague Trockenbeerenausle...,NaN,Chardonnay,2015.0,"Burgenland, Austria"
1,10.50%,A touch of creaminess on the nose leads to a m...,H.& C. Nittnaus 2015 Trockenbeerenauslese Gran...,$45,Chardonnay,2015.0,"Burgenland, Austria"
2,9%,A smokiness lies thickly above the apple fruit...,Rosenhof 2015 Trockenbeerenauslese Chardonnay ...,$29,Chardonnay,2015.0,"Burgenland, Austria"
3,10%,"Lime blossom tea, candied apple and an allurin...",Gunter Triebaumer 2015 Beerenauslese Chardonna...,$25,Chardonnay,2015.0,"Burgenland, Austria"
4,10.50%,"Stone, earth and pear show on the nose. The pa...",Gunter Triebaumer 2016 CuvÃ©e SpÃ¤tlese Chardo...,$15,Chardonnay,2016.0,"Burgenland, Austria"


In [30]:
variety_geos = wine_df_merged.groupby(['Variety', 'geo_normalized']).size()
at_least_n_types = variety_geos[variety_geos > 30].reset_index()
wine_df_merged_filtered = pd.merge(wine_df_merged, at_least_n_types, left_on=['Variety', 'geo_normalized'], right_on=['Variety', 'geo_normalized'])
wine_df_merged_filtered = wine_df_merged_filtered[['Name', 'Variety', 'geo_normalized', 'Description']]
print(wine_df_merged_filtered.shape)

(11499, 4)


In [31]:
wine_reviews = list(wine_df_merged_filtered['Description'])

descriptor_mapping = pd.read_csv('descriptor_mapping_tastes.csv', encoding='latin1').set_index('raw descriptor')

core_tastes = ['aroma', 'weight', 'sweet', 'acid', 'salt', 'piquant', 'fat', 'bitter']
descriptor_mappings = dict()
for c in core_tastes:
    if c=='aroma':
        descriptor_mapping_filtered=descriptor_mapping.loc[descriptor_mapping['type']=='aroma']
    else:
        descriptor_mapping_filtered=descriptor_mapping.loc[descriptor_mapping['primary taste']==c]
    descriptor_mappings[c] = descriptor_mapping_filtered                                                   
    

def return_descriptor_from_mapping(descriptor_mapping, word, core_taste):
    if word in list(descriptor_mapping.index):
        descriptor_to_return = descriptor_mapping['combined'][word]
        return descriptor_to_return
    else:
        return None

review_descriptors = []
for review in wine_reviews:
    taste_descriptors = []
    normalized_review = normalize_text(review)
    phrased_review = wine_trigram_model[normalized_review]
#     print(phrased_review)
    
    for c in core_tastes:                                                      
        descriptors_only = [return_descriptor_from_mapping(descriptor_mappings[c], word, c) for word in phrased_review]
        no_nones = [str(d).strip() for d in descriptors_only if d is not None]
        descriptorized_review = ' '.join(no_nones)
        taste_descriptors.append(descriptorized_review)
    review_descriptors.append(taste_descriptors)

In [32]:
print(phrased_review)

['sick', 'sweet', 'vanilla', 'oak', 'temper', 'light', 'tart', 'pineappl', 'fruit', 'ultim', 'come', 'across', 'simpl', 'onedimension']


In [33]:
review_descriptors

[['marmalade toffee vanilla citrus candy orange_peel candy orange citrus vanilla',
  'full_bodied',
  'dry',
  'high_acid high_acid',
  '',
  '',
  '',
  'low_tannin'],
 ['lactic honey baked_apple fruit vanilla lemon lactic',
  '',
  '',
  'high_acid high_acid high_acid high_acid',
  '',
  '',
  'lactic lactic',
  ''],
 ['smoke apple fruit smoke lemon apricot plum fruit',
  'full_bodied full_bodied',
  '',
  'high_acid high_acid high_acid',
  '',
  '',
  '',
  ''],
 ['lime tea candy apple smoke botrytis spice',
  '',
  'sweet sweet',
  'high_acid high_acid',
  '',
  '',
  '',
  ''],
 ['stone earth pear pear orange rind salinity cheese',
  'light_bodied',
  'dry dry off_dry',
  '',
  'salinity',
  '',
  '',
  'high_tannin'],
 ['tea candy lemon spice cheese',
  '',
  'sweet very_sweet sweet',
  'high_acid',
  '',
  '',
  '',
  ''],
 ['hazelnut vanilla honey', '', '', '', '', '', '', ''],
 ['lemon honey minerality',
  'full_bodied',
  'sweet',
  'high_acid high_acid',
  '',
  '',
  '',
  

In [34]:
taste_descriptors = []
taste_vectors = []

for n, taste in enumerate(core_tastes):
    print(taste)
    taste_words = [r[n] for r in review_descriptors]
    
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit(taste_words)
    dict_of_tfidf_weightings = dict(zip(X.get_feature_names(), X.idf_))
        
    wine_review_descriptors = []
    wine_review_vectors = []
    
    for d in taste_words:
        descriptor_count = 0
        weighted_review_terms = []
        terms = d.split(' ')
        for term in terms:
            if term in dict_of_tfidf_weightings.keys():
                tfidf_weighting = dict_of_tfidf_weightings[term]
                try:
                    word_vector = wine_word2vec_model.wv.get_vector(term).reshape(1, 300)
                    weighted_word_vector = tfidf_weighting * word_vector
                    weighted_review_terms.append(weighted_word_vector)
                    descriptor_count += 1
                except:
                    continue
            else:
                continue
        try:
            review_vector = sum(weighted_review_terms)/len(weighted_review_terms)
            review_vector = review_vector[0]
        except:
            review_vector = np.nan
#         terms_and_vec = [terms, review_vector]
        wine_review_vectors.append(review_vector)
        wine_review_descriptors.append(terms)
    
    taste_vectors.append(wine_review_vectors)
    taste_descriptors.append(wine_review_descriptors)
    

taste_vectors_t = list(map(list, zip(*taste_vectors)))
taste_descriptors_t = list(map(list, zip(*taste_descriptors)))

review_vecs_df = pd.DataFrame(taste_vectors_t, columns=core_tastes)

columns_taste_descriptors = [a + '_descriptors' for a in core_tastes]
review_descriptors_df = pd.DataFrame(taste_descriptors_t, columns=columns_taste_descriptors)

wine_df_vecs = pd.concat([wine_df_merged_filtered, review_descriptors_df, review_vecs_df], axis=1)
wine_df_vecs.head(5)

aroma


C:\Users\Boss Man\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


weight
sweet
acid
salt
piquant
fat
bitter


,Name,Variety,geo_normalized,Description,aroma_descriptors,weight_descriptors,sweet_descriptors,acid_descriptors,salt_descriptors,piquant_descriptors,fat_descriptors,bitter_descriptors,aroma,weight,sweet,acid,salt,piquant,fat,bitter
0,Kracher 2015 Nouvelle Vague Trockenbeerenausle...,Chardonnay,"Burgenland, Austria","Marmalade, toffee, vanilla and tonka bean swir...","[marmalade, toffee, vanilla, citrus, candy, or...",[full_bodied],[dry],"[high_acid, high_acid]",[],[],[],[low_tannin],"[-0.80917645, -2.9146504, 3.098291, -0.5997139...","[-3.5445297, -3.1590269, 2.1393247, -0.1505228...","[-0.9271003, 1.3765962, 5.127116, -0.65254116,...","[-0.3217097, -0.32687074, 0.28517705, -0.10552...",NaN,NaN,NaN,NaN
1,H.& C. Nittnaus 2015 Trockenbeerenauslese Gran...,Chardonnay,"Burgenland, Austria",A touch of creaminess on the nose leads to a m...,"[lactic, honey, baked_apple, fruit, vanilla, l...",[],[],"[high_acid, high_acid, high_acid, high_acid]",[],[],"[lactic, lactic]",[],"[-0.4795868, -0.4251326, 1.4899663, 0.6480437,...",NaN,NaN,"[-0.3217097, -0.32687074, 0.28517705, -0.10552...",NaN,NaN,"[-1.0568256, 0.047783077, 0.85501873, 0.079494...",NaN
2,Rosenhof 2015 Trockenbeerenauslese Chardonnay ...,Chardonnay,"Burgenland, Austria",A smokiness lies thickly above the apple fruit...,"[smoke, apple, fruit, smoke, lemon, apricot, p...","[full_bodied, full_bodied]",[],"[high_acid, high_acid, high_acid]",[],[],[],[],"[-1.0546242, -1.262496, 2.4463913, 0.7602774, ...","[-3.5445297, -3.1590269, 2.1393247, -0.1505228...",NaN,"[-0.3217097, -0.32687074, 0.28517705, -0.10552...",NaN,NaN,NaN,NaN
3,Gunter Triebaumer 2015 Beerenauslese Chardonna...,Chardonnay,"Burgenland, Austria","Lime blossom tea, candied apple and an allurin...","[lime, tea, candy, apple, smoke, botrytis, spice]",[],"[sweet, sweet]","[high_acid, high_acid]",[],[],[],[],"[-1.9449217, -0.8668419, 4.2323275, 3.0929484,...",NaN,"[-5.728736, -3.1177607, 6.1547, -3.7910526, -2...","[-0.3217097, -0.32687074, 0.28517705, -0.10552...",NaN,NaN,NaN,NaN
4,Gunter Triebaumer 2016 CuvÃ©e SpÃ¤tlese Chardo...,Chardonnay,"Burgenland, Austria","Stone, earth and pear show on the nose. The pa...","[stone, earth, pear, pear, orange, rind, salin...",[light_bodied],"[dry, dry, off_dry]",[],[salinity],[],[],[high_tannin],"[-0.50347614, -0.19549707, 1.3448436, -2.07301...","[-1.3933558, -2.2020807, 1.7833804, -0.4134141...","[-0.9271003, 1.3765962, 5.127116, -0.65254116,...",NaN,NaN,NaN,NaN,NaN


In [35]:
# pull the average embedding for the wine attribute across all wines. 
avg_taste_vecs = dict()
for t in core_tastes:
    # look at the average embedding for a taste, across all wines that have descriptors for that taste 
    review_arrays = wine_df_vecs[t].dropna()
    average_taste_vec = np.average(review_arrays)
    avg_taste_vecs[t] = average_taste_vec

C:\Users\Boss Man\anaconda3\lib\site-packages\numpy\lib\function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
C:\Users\Boss Man\anaconda3\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [36]:
avg_taste_vecs

{'aroma': array([-1.3431231 , -0.5166458 ,  2.6314423 , -0.03276914,  1.9485685 ,
        -2.3252347 , -2.4314327 ,  0.37031946,  1.2940059 ,  0.5294585 ,
        -2.0282829 ,  0.19435167, -0.41349366,  0.68541026,  0.573247  ,
         0.66862845,  1.7930257 ,  3.5299606 ,  2.2058578 , -1.3302691 ,
        -1.2052372 , -2.371138  , -0.31305817, -1.6949297 , -2.454377  ,
         0.16195303, -4.8064933 ,  1.8739612 , -1.8872765 ,  1.1097857 ,
         0.929393  , -3.5361173 , -2.2724802 ,  0.7685178 , -0.03686303,
        -0.17830746, -3.1968808 ,  1.243365  ,  1.9957899 ,  1.816531  ,
        -0.7620247 , -1.2204909 , -1.1520238 , -1.8085907 ,  0.75410044,
        -0.5920568 , -0.886283  ,  0.7543871 ,  0.4119393 ,  0.4736142 ,
        -2.278439  ,  0.19684625, -0.53198624, -1.1237136 , -0.25784206,
        -1.3814808 , -1.0355844 ,  1.4999685 , -0.9944461 ,  0.8360587 ,
        -0.14874397, -1.954622  , -0.8184823 ,  2.0527816 ,  0.6935166 ,
        -2.7251217 ,  1.5918363 ,  0.61725

In [37]:
normalized_geos = list(set(zip(wine_df_vecs['Variety'], wine_df_vecs['geo_normalized'])))

def subset_wine_vectors(list_of_varieties, wine_attribute):
    wine_variety_vectors = []
    for v in list_of_varieties:

        one_var_only = wine_df_vecs.loc[(wine_df_vecs['Variety'] == v[0]) & 
                                                (wine_df_vecs['geo_normalized'] == v[1])]
        if len(list(one_var_only.index)) < 1 or str(v[1][-1]) == '0':
            continue
        else:
            taste_vecs = list(one_var_only[wine_attribute])
            taste_vecs = [avg_taste_vecs[wine_attribute] if 'numpy' not in str(type(x)) else x for x in taste_vecs]
            average_variety_vec = np.average(taste_vecs, axis=0)
            
            descriptor_colname = wine_attribute + '_descriptors'
            all_descriptors = [i[0] for i in list(one_var_only[descriptor_colname])]
            word_freqs = Counter(all_descriptors)
            most_common_words = word_freqs.most_common(50)
            top_n_words = [(i[0], "{:.2f}".format(i[1]/len(taste_vecs))) for i in most_common_words]
            top_n_words = [i for i in top_n_words if len(i[0])>2]
            wine_variety_vector = [v, average_variety_vec, top_n_words]
                
            wine_variety_vectors.append(wine_variety_vector)
            
    return wine_variety_vectors

In [65]:
def pca_wine_variety(list_of_varieties, wine_attribute, pca=True):
    wine_var_vectors = subset_wine_vectors(normalized_geos, wine_attribute)
    
    wine_varieties = [str(w[0]).replace('(', '').replace(')', '').replace("'", '').replace('"', '') for w in wine_var_vectors]
    wine_var_vec = [w[1] for w in wine_var_vectors]
    if pca:
        pca = PCA(n_components=1)
        wine_var_vec = pca.fit_transform(wine_var_vec)
        wine_var_vec = pd.DataFrame(wine_var_vec, index=wine_varieties)
    else:
        wine_var_vec = pd.Series(wine_var_vec, index=wine_varieties)
    wine_var_vec.sort_index(inplace=True)
    
    wine_descriptors = pd.DataFrame([w[2] for w in wine_var_vectors], index=wine_varieties)
    wine_descriptors = pd.melt(wine_descriptors.reset_index(), id_vars='index')
    wine_descriptors.sort_index(inplace=True)
    
    return wine_var_vec, wine_descriptors
    

In [85]:
all_nonaromas.isnull().sum()

aroma       0
weight      0
sweet       0
acid        0
salt       49
piquant     0
fat         0
bitter     49
dtype: int64

In [76]:
all_nonaromas.shape

(49, 8)

In [124]:
taste_dataframes =[]
# generate the dataframe of aromas vectors as output, 
aroma_vec, aroma_descriptors = pca_wine_variety(normalized_geos, 'aroma', pca=False)
taste_dataframes.append(aroma_vec)

aroma_vec, nonaroma_descriptors = pca_wine_variety(normalized_geos, 'weight', pca=True)
taste_dataframes.append(aroma_vec)

aroma_vec, nonaroma_descriptors = pca_wine_variety(normalized_geos, 'sweet', pca=True)
taste_dataframes.append(aroma_vec)

aroma_vec, nonaroma_descriptors = pca_wine_variety(normalized_geos, 'acid', pca=True)
taste_dataframes.append(aroma_vec)

aroma_vec, nonaroma_descriptors = pca_wine_variety(normalized_geos, 'salt', pca=False)
taste_dataframes.append(aroma_vec)

aroma_vec, nonaroma_descriptors = pca_wine_variety(normalized_geos, 'piquant', pca=True)
taste_dataframes.append(aroma_vec)

aroma_vec, nonaroma_descriptors = pca_wine_variety(normalized_geos, 'fat', pca=True)
taste_dataframes.append(aroma_vec)

aroma_vec, nonaroma_descriptors = pca_wine_variety(normalized_geos, 'bitter', pca=False)
taste_dataframes.append(aroma_vec)

    
# combine all the dataframes created above into one 
all_nonaromas = pd.concat(taste_dataframes, axis=1)
all_nonaromas.columns = core_tastes

In [125]:
all_nonaromas

,aroma,weight,sweet,acid,salt,piquant,fat,bitter
"Chardonnay, Auckland, New Zealand","[-1.1107708, -0.64572626, 2.3254745, 0.1304707...",-4.363866,0.455521,-2.040361e-05,NaN,-0.000072,9.749360e-06,NaN
"Chardonnay, Beaujolais, France","[-1.4559786, -0.8481796, 1.9362681, -0.4466512...",0.574543,0.149068,3.830368e-05,NaN,-0.000073,-2.427107e-06,NaN
"Chardonnay, Breede River Valley","[-1.8132936, -0.7632356, 2.3815877, -0.0889262...",0.396770,-2.900181,4.282272e-05,NaN,-0.000073,3.018082e-06,NaN
"Chardonnay, British Columbia, Canada","[-1.3201736, -0.10929091, 2.430613, 0.63138866...",5.306101,0.149070,-2.863626e-05,NaN,-0.000071,-4.092883e-05,NaN
"Chardonnay, Burgenland, Austria","[-0.9049106, -0.8054461, 1.9607682, 0.17470571...",-2.363968,3.060064,5.477734e-06,NaN,-0.000073,-1.892178e-06,NaN
"Chardonnay, Cape South Coast, South Africa","[-1.452805, -0.80962485, 2.1470902, 0.31894487...",-0.852390,-4.373455,1.849066e-05,NaN,-0.000073,-1.997977e-05,NaN
"Chardonnay, Casablanca Valley, Aconcagua Region, Chile","[-1.371048, -0.87995696, 2.6637473, 0.24838689...",-2.028589,-4.756800,-1.737156e-05,NaN,-0.000074,3.020534e-06,NaN
"Chardonnay, Chablis, Burgundy, France","[-1.6719638, -0.43834072, 2.4395957, -0.567167...",-1.681513,-0.022237,2.262711e-05,NaN,0.000540,7.969657e-05,NaN
"Chardonnay, Champagne, France","[-1.8552374, -0.2417804, 2.5501108, -0.9778119...",-0.161466,6.830817,3.545344e-05,NaN,0.000018,1.719439e-05,NaN
"Chardonnay, Clear Lake, North Coast, California","[-1.3022314, -0.1818406, 2.1652057, 0.5451122,...",-5.599400,-4.898093,-1.812708e-05,NaN,-0.000071,-5.655342e-05,NaN


In [126]:
aroma_descriptors_copy = aroma_descriptors.copy()
aroma_descriptors_copy.set_index('index', inplace=True)
aroma_descriptors_copy.dropna(inplace=True)

aroma_descriptors_copy = pd.DataFrame(aroma_descriptors_copy['value'].tolist(), index=aroma_descriptors_copy.index)
aroma_descriptors_copy.columns = ['descriptors', 'relative_frequency']
aroma_descriptors_copy.to_csv('wine_varietyy_descriptors.csv')

In [127]:
aroma_descriptors_copy.head()

,descriptors,relative_frequency
index,,
"Chardonnay, Colchagua Valley, Central Valley, Chile",oak,0.09
"Chardonnay, Marlborough, New Zealand",toast,0.15
"Chardonnay, Columbia Valley, Washington, USA",fruit,0.13
"Chardonnay, Columbia Valley, Oregon, USA",fruit,0.29
"Chardonnay, Maipo Valley, Central Valley, Chile",apple,0.13


In [128]:
aroma_descriptors_copy.tail()

,descriptors,relative_frequency
index,,
"Chardonnay, Monterey, Central Coast, California, USA",lime_peel,0.00
"Chardonnay, Napa-Sonoma, North Coast, California, USA",orange,0.00
"Chardonnay, Finger Lakes, New York, USA",perfumed,0.00
"Chardonnay, San Luis Obispo, Central Coast, California, USA",melon,0.01
"Chardonnay, Sonoma Valley, North Coast, California, USA",new_oak,0.00


In [132]:
aroma_descriptors_copy['descriptors'].value_counts()

fruit       47
toast       47
oak         46
lactic      46
ripe        46
            ..
coffee       1
nettle       1
mushroom     1
cucumber     1
truffle      1
Name: descriptors, Length: 194, dtype: int64

In [131]:
aroma_descriptors_copy[aroma_descriptors_copy['descriptors']=='salinity']

,descriptors,relative_frequency
index,,
"Chardonnay, San Antonio, Aconcagua Region, Chile",salinity,0.10
"Chardonnay, San Luis Obispo, Central Coast, California, USA",salinity,0.07
"Chardonnay, Santa Barbara, Central Coast, California, USA",salinity,0.04
"Chardonnay, Sonoma Valley, North Coast, California, USA",salinity,0.05
"Chardonnay, Napa-Sonoma, North Coast, California, USA",salinity,0.04
"Chardonnay, Casablanca Valley, Aconcagua Region, Chile",salinity,0.03
"Chardonnay, Auckland, New Zealand",salinity,0.04
"Chardonnay, Napa Valley, North Coast, California, USA",salinity,0.03
"Chardonnay, Monterey, Central Coast, California, USA",salinity,0.02


In [129]:
def normalize(df, cols_to_normalize):
    for feature_name in cols_to_normalize:
        print(feature_name)
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        df[feature_name] = df[feature_name].apply(lambda x: (x- min_value)/(max_value-min_value))
#         (df[feature_name] - min_value) / (max_value - min_value)
    return df

all_nonaromas_normalized = normalize(all_nonaromas, cols_to_normalize=['weight', 'sweet', 'salt', 'bitter', 'acid', 'piquant', 'fat'])
all_nonaromas_normalized.to_csv('wine_aromass_nonaromas.csv')

weight
sweet
salt
bitter
acid
piquant
fat


# Preparing The Food Dataset

In [114]:
foods = pd.read_csv('list_of_foods.csv')
foods_list = list(foods['Food'])
foods_list_normalized = [normalize_text(f) for f in foods_list]
foods_list_preprocessed = [food_trigram_model[f][0] for f in foods_list_normalized]
foods_list_preprocessed = list(set(foods_list_preprocessed))

In [115]:
foods_vecs = dict()

word_vectors = wine_word2vec_model.wv
for f in foods_list_preprocessed:
    try:
        food_vec = word_vectors[f]
        foods_vecs[f] = food_vec
    except:
        continue


In [116]:
from scipy import spatial

core_tastes_revised = {'weight': ['heavy', 'full_bodied', 'thick', 'milk', 'fat', 'steak', 'bold', 'pizza', 'pasta', 'creamy', 'bread'],
                       'sweet': ['sweet', 'sugar', 'cake', 'mango', 'stevia'], 
                       'acid': ['acid', 'sour', 'vinegar', 'yoghurt'],
                       'salt': ['salty', 'salty', 'parmesan', 'oyster', 'pizza', 'bacon', 'potato_chip'], 
                       'piquant': ['spicy'], 
                       'fat': ['fat', 'fried', 'creamy', 'cake', 'brie'], 
                       'bitter': ['bitter', 'kale']
                      }

average_taste_vecs = dict()
core_tastes_distances = dict()
for taste, keywords in core_tastes_revised.items():
    
    all_keyword_vecs = []
    for keyword in keywords:
        c_vec = word_vectors[keyword]
        all_keyword_vecs.append(c_vec)
    
    avg_taste_vec = np.average(all_keyword_vecs, axis=0)
    average_taste_vecs[taste] = avg_taste_vec
        
    taste_distances = dict()
    for k, v in foods_vecs.items():
        similarity = 1- spatial.distance.cosine(avg_taste_vec, v)
        taste_distances[k] = similarity
        
    core_tastes_distances[taste] = taste_distances  

In [117]:
food_nonaroma_infos = dict()
# for each core taste, identify the food item that is farthest and closest. We will need this to create a normalized scale between 0 and 1
for key, value in core_tastes_revised.items():
    dict_taste = dict()
    farthest = min(core_tastes_distances[key], key=core_tastes_distances[key].get)
    farthest_distance = core_tastes_distances[key][farthest]
    closest = max(core_tastes_distances[key], key=core_tastes_distances[key].get)
    closest_distance = core_tastes_distances[key][closest]
    print(key, farthest, closest)
    dict_taste['farthest'] = farthest_distance
    dict_taste['closest'] = closest_distance
    dict_taste['average_vec'] = average_taste_vecs[key]
    food_nonaroma_infos[key] = dict_taste

weight rabbit bread
sweet filet mango
acid brussel tart
salt black_tea bacon
piquant sunda wasabi
fat foodstuff cake
bitter quail tart


In [118]:
food_nonaroma_infos_df = pd.DataFrame(food_nonaroma_infos).T
food_nonaroma_infos_df.to_csv('average_nonaromaa_vectors.csv')

In [120]:
food_nonaroma_infos_df

,farthest,closest,average_vec
weight,-0.15502,0.504727,"[-0.107819535, -0.428998, 0.17753184, 0.455737..."
sweet,-0.283566,0.512151,"[0.49864894, -0.8851121, 0.12699969, -0.097148..."
acid,-0.176347,0.5793,"[-0.3244881, -0.16371989, 1.3374262, 0.2912337..."
salt,-0.176218,0.635676,"[-0.2548894, -0.06102786, 0.3053675, 0.2184848..."
piquant,-0.12478,0.389423,"[-0.20364383, 0.06061496, 0.1920985, 0.0926313..."
fat,-0.117658,0.610474,"[0.20618415, -0.02715757, 0.26209885, 0.306617..."
bitter,-0.166944,0.432018,"[-0.38733697, -0.33273724, 1.2993782, 0.659761..."
